This notebook is meant to explore and check the correspondence of the SimChannel data with the wire images.

Correct information is required to provide truth labels.

In [ ]:
import ROOT as rt
import numpy as np
import plotly as pl
import plotly.graph_objects as go
from larcv import larcv
from larlite import larlite
import lardly
%load_ext autoreload 
%autoreload 2

In [ ]:
from lardly.detectoroutline import get_tpc_boundary_plot
from larlite import larutil

#larutil.LArUtilConfig.SetDetector( larlite.geo.kSBND )
larutil.LArUtilConfig.SetDetector( larlite.geo.kMicroBooNE )
NTPCS=1

TPC_PLOT = {}
TPC_TICK_PLOT = {}
for ITPC in range(NTPCS):
    TPC_PLOT[ITPC] = get_tpc_boundary_plot(cryoid=0,tpcid=ITPC)
    TPC_TICK_PLOT[ITPC] = get_tpc_boundary_plot(cryoid=0,tpcid=ITPC,use_tick_dimensions=True)

driftv = larutil.LArProperties.GetME().DriftVelocity()
cm_per_tick = driftv*0.5

In [ ]:
infile = rt.TFile("testout_simchannelvoxelizer.root")
simchvox = infile.Get("simchannelvoxelizer")
nentries = simchvox.GetEntries()


ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename( "dlmerged.root")
ioll.open()

iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickForward )
iolcv.add_in_file( "dlmerged.root")
iolcv.initialize()

In [ ]:
IENTRY = 0
simchvox.GetEntry(IENTRY)
ioll.go_to(IENTRY)
iolcv.read_entry(IENTRY)

In [ ]:
# sync ancestor ID colors
ancestor_color = {}
for ITPC in range(NTPCS):
    ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()
    ancestor_list = np.unique(ancestor)
    for aid in ancestor_list:
        if aid not in ancestor_color:
            xcolor = np.zeros(3,dtype=np.dtype)
            xcolor[0] = np.random.randint(0,255)
            xcolor[1] = np.random.randint(0,255)
            xcolor[2] = np.random.randint(0,255)
            ancestor_color[aid] = "rgb(%d,%d,%d)"%(xcolor[0],xcolor[1],xcolor[2])
            print(aid,": ",ancestor_color[aid])

In [ ]:
# make bank images for the TPC, fill them using the tick information from the voxelized simchannel info.
# we then can contrast this with remaining wire charge we cannot truth-associate 
# or find truth-labeled locations with no simulated wire signal (WTF)

ITPC = 0





In [ ]:
event_mctrack  = ioll.get_data( larlite.data.kMCTrack,  "mcreco" )
event_mcshower = ioll.get_data( larlite.data.kMCShower, "mcreco" )

print("Number of mctracks: ",event_mctrack.size())
print("Number of mcshowers: ",event_mcshower.size())


In [ ]:
apply_sce = False

plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=apply_sce,
                                                             no_offset=True )

plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=True,
                                                              apply_sce=apply_sce,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)

plot_mctracks_sce  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=True,
                                                             no_offset=True )

plot_mcshowers_sce = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=True,
                                                              apply_sce=True,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)

In [ ]:
# PLOT TRUE POSITIONS

plot_list = []

for ITPC in range(NTPCS):
    coord  = simchvox.coordindex_v.at(ITPC).tonumpy()
    pos    = simchvox.truepos_v.at(ITPC).tonumpy()
    ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()

    ancestor_list = np.unique(ancestor)
    
    plot_list.append( TPC_PLOT[ITPC] )
    
    #if ITPC==0:
    #    continue
    
    for aid in ancestor_list:
        if aid==0:
            continue
            
        zcolor = ancestor_color[aid]
        xpos = pos[ ancestor[:]==aid, : ].astype(np.float32)
        
        simchvoxels = {
            "type":"scatter3d",
            "x": xpos[:,0],
            "y": xpos[:,1],
            "z": xpos[:,2],
            "mode":"markers",
            "name":"AID[%d]"%(aid),
            "marker":{"color":zcolor,"size":1,"opacity":0.8,"colorscale":'Viridis'},
        }

        plot_list.append( simchvoxels )
        

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    width=1000,
    height=1000,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 1},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# PLOT TICK POSITIONS
TICKMIN=0
TICKMAX=10000

plot_list = []

for ITPC in range(NTPCS):
    coord  = simchvox.coordindex_v.at(ITPC).tonumpy()
    pos    = simchvox.truepos_v.at(ITPC).tonumpy()
    ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()

    ancestor_list = np.unique(ancestor)
    
    for aid in ancestor_list:
        if aid==0:
            continue
            
        xcoord = coord[ ancestor[:]==aid, : ].astype(np.float32)
        
        # we enforce min and max
        xcut = xcoord[ xcoord[:,0]>TICKMIN, : ]
        xcut = xcut[ xcut[:,0]<TICKMAX, : ]
        
        zcolor = ancestor_color[aid]
        
        #if ITPC==0:
        #    xcut[:,0] *= -1
        xcut[:,0] -= 3200-2400
        xcut[:,0] *= cm_per_tick
        xcut[:,1] *= 0.3
        xcut[:,1] -= 116.02999877929688
        xcut[:,2] *= 0.3
        
        simchvoxels = {
            "type":"scatter3d",
            "x": xcut[:,0],
            "y": xcut[:,1],
            "z": xcut[:,2],
            "mode":"markers",
            "name":"AID[%d]"%(aid),
            "marker":{"color":zcolor,"size":1,"opacity":0.8,"colorscale":'Viridis'},
        }

        plot_list.append( simchvoxels )
        plot_list.append( TPC_PLOT[ITPC] )
        plot_list += plot_mctracks_sce
        plot_list += plot_mcshowers_sce

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    width=1000,
    height=1000,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 1},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# Get the images from the tree storing the wireplane images

# we retrieve a container with all the wireplane images for the event
event_image_data = iolcv.get_data( larcv.kProductImage2D, "wiremc" )

# we get a std::vector<larcv::Image2D> object that has the data
adc_v = event_image_data.as_vector()

# print the number of images in the vector
# usually, if this is a MicroBooNE file you should get 3 images: one for each wireplane U, V, and Y.
print("number of images in this event: ",adc_v.size())

In [ ]:
# plot the first image using plotly: takes a few seconds to load

PLANE = 2 # options are 0,1, or 2 for the U,V,Y plane
plane_image2 = adc_v.at(PLANE)
plane_plot2 = lardly.data.visualize_larcv_image2d( plane_image2, reverse_ticks=False, maxz=100.0 )

PLANE = 0 # options are 0,1, or 2 for the U,V,Y plane
plane_image5 = adc_v.at(PLANE)
plane_plot5 = lardly.data.visualize_larcv_image2d( plane_image5, reverse_ticks=False, maxz=100.0 )

# plotly figure

fig5 = go.Figure( data=[plane_plot5] )
fig5.show()

fig2 = go.Figure( data=[plane_plot2] )
fig2.show()



## If opening the tutorial file, merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root,
## the image should be of a Charged-Current Electron Neutrino Interaction
## If on the U-plane, the neutrino interaction vertex is around (1650,5328)